
# PDEs and finite difference Methods | Week 02

This notes are taken from
> Kværnø, A. (2020, November 4). *Partial differential equations and finite difference methods*. Lecture notes for TMA4125/4130/4135 Mathematics 4N/D. Norwegian University of Science and Technology (NTNU). [https://wiki.math.ntnu.no/_media/tma4130/2020h/pde.pdf](https://wiki.math.ntnu.no/_media/tma4130/2020h/pde.pdf)

## Introduction

In this note the finite difference method for solving partial differential equations (PDEs) will be presented.

Roughly speaking, a finite difference method consists of the following steps:

1. Discretize the domain on which the equation is defined.
2. On each grid point, replace the derivatives with an approximation, using the values in neighbouring grid points.
3. Replace the exact solutions by their approximations.
4. Solve the resulting system of equations.

We will first see how to find approximations to the derivative of a function, and then how these can be used to solve boundary value problems like

$$u'' + p(x)u' + q(x)u = r(x), \, a \leq x \leq b, \, u(a) = u_a, \,  u(b) = u_b,$$

and time dependent partial differential equations like the heat equation

$$u_t = u_{xx}.$$

The technique described here is, however, applicable to several other PDEs, and it is therefore important to try to understand the underlying idea.

## Numerical differentiation

This is the main tool for finite difference methods.

Given a sufficiently smooth function f. How can we find an approximation to f'(x) or f''(x) in some given point x, just by using evaluation of the function itself?

The derivative of f is defined as

$$f'(x) = \lim_{h\to 0} \frac{f(x+h) - f(x)}{h}.$$

Given a sufficiently small value of h, the right hand side can be used an approximation to f'(x). A small collection of the most used approximations to f'(x) is:

$$f'(x) \approx \begin{cases}
\frac{f(x+h) - f(x)}{h}, & \text{Forward difference}, \\ \frac{f(x) - f(x-h)}{h}, & \text{Backward difference}, \\ \frac{f(x+h) - f(x-h)}{2h}, & \text{Central difference}. \end{cases}$$

The first one is taken directly from the definition, and so is the second, and the third is just the mean of the first two. A common approximation to the second derivative is

$$f''(x) \approx \frac{f(x+h) - 2f(x) + f(x-h)}{h^2}.$$

### Numerical example 1:

Test the method on the function f(x) = sin(x) at the point x = π/4. Compare with the exact derivative. Try different step sizes, e.g. h = 0.1, h = 0.01, h = 0.001. Notice how the error in each case changes with h.

In [3]:
# Numerical differentiation

from math import sin, cos, pi


# Forward difference
def diff_forward(f, x, h=0.1):
    return (f(x + h) - f(x)) / h


# Backward difference
def diff_backward(f, x, h=0.1):
    return (f(x) - f(x - h)) / h


# Central difference for f'(x):
def diff_central(f, x, h=0.1):
    return (f(x + h) - f(x - h)) / (2 * h)


# end of diff_central
# Central difference for f''(x):
def diff2_central(f, x, h=0.1):
    return (f(x + h) - 2 * f(x) + f(x - h)) / h**2


# end of diff2_central
# Numerical example 1
x = pi / 4
df_exact = cos(x)
ddf_exact = -sin(x)
h = 0.1
f = sin
df = diff_forward(f, x, h)
print("Approximations to the first derivative")
print("Forward difference: df = {:12.8f}, Error = {:10.3e} ".format(df, df_exact - df))
df = diff_backward(f, x, h)
print("Backward difference: df = {:12.8f}, Error = {:10.3e} ".format(df, df_exact - df))
df = diff_central(f, x, h)
print("Central difference: df = {:12.8f}, Error = {:10.3e} ".format(df, df_exact - df))
print("Approximation to the second derivative")
ddf = diff2_central(f, x, h)
print(
    "Central difference: ddf= {:12.8f}, Error = {:10.3e} ".format(ddf, ddf_exact - ddf)
)

Approximations to the first derivative
Forward difference: df =   0.67060297, Error =  3.650e-02 
Backward difference: df =   0.74125475, Error = -3.415e-02 
Central difference: df =   0.70592886, Error =  1.178e-03 
Approximation to the second derivative
Central difference: ddf=  -0.70651772, Error = -5.891e-04 


### Error analysis

In this case the error analysis is quite simple: Perform a Taylor expansion of the error around x. The Taylor expansion becomes a power series in h.

The expansion for the error of the forward difference is:

$$\begin{align*} e(x; h) &= f'(x) - \frac{f(x+h) - f(x)}{h} \\ &= f'(x) - \frac{(f(x) + f'(x)h + \frac{1}{2}f''(\xi)h^2) - f(x)}{h} \\ &= -\frac{1}{2}f''(\xi)h \end{align*}$$

for some $\xi \in (x, x+h)$, $

The expansion for the error of the central difference is slightly more complicated:

$$\begin{align*} e(x; h) &= f'(x) - \frac{f(x+h) - f(x-h)}{2h} \\ &= f'(x) - \frac{(f(x) + f'(x)h + \frac{1}{2}f''(x)h^2 + \frac{1}{6}f'''(\xi_1)h^3) - (f(x) - f'(x)h + \frac{1}{2}f''(x)h^2 - \frac{1}{6}f'''(\xi_2)h^3)}{2h}\\ &= -\frac{1}{12}(f'''(\xi_1) + f'''(\xi_2))h^2\\ &= -\frac{1}{6}f'''(\eta)h^2 \end{align*}$$

for some $\eta \in (x-h,x+h)$.

In the last step, the two remainder terms have been combined by the intermediate value theorem (Result 2 at the end of Preliminaries). The error for the approximation of the second order derivative can be found in a similar manner.

The order of an approximation is p if there exist a constant C independent on h such that

$$|e(h; x)| \leq Ch^p$$

for all sufficiently small h > 0, see Preliminaries.

In practice, it is sufficient to show that the power expansion of the error satisfies

$$e(x, h) = C_ph^p + C_{p+1}h^{p+1} + \cdots, \text{ with } C_p \neq 0$$

The forward and backward approximations are of order 1, the central differences of order 2.

We are going to use these formulas a lot in the sequel, so let us just summarize the results, including the error terms:

**Difference formulas for derivatives:**

$$\begin{align*} f'(x) &= \begin{cases} \frac{f(x+h) - f(x)}{h} - \frac{h}{2}f''(\xi), & \text{Forward difference}\\ \frac{f(x) - f(x-h)}{h} + \frac{h}{2}f''(\xi), & \text{Backward difference}\\ \frac{f(x+h) - f(x-h)}{2h} - \frac{h^2}{6}f'''(\xi). & \text{Central difference} \end{cases}\\ f''(x) &= \frac{f(x+h) - 2f(x) + f(x-h)}{h^2} - \frac{h^2}{12}f^{(4)}(\xi), \quad \text{Central difference} \end{align*}$$